In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://xps13.lan:4040
SparkContext available as 'sc' (version = 2.3.1, master = local[*], app id = local-1543439624774)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@21e3eb6e


In [2]:
spark.version

res1: String = 2.3.1


# Kaggle Dataset Load

# DoE Dataset Load concise

In [3]:
val inHeader = sc.textFile("MERGED2016_17_PP.csv").map(x => x + ",YEAR").map(line =>line.split(","))

val in2016 = sc.textFile("MERGED2016_17_PP.csv").map(x => x + ",2016").map(line =>line.split(","))

val in2015 = sc.textFile("MERGED2015_16_PP.csv").map(x => x + ",2015").map(line =>line.split(","))

val in2014 = sc.textFile("MERGED2014_15_PP.csv").map(x => x + ",2014").map(line =>line.split(","))

val in2013 = sc.textFile("MERGED2013_14_PP.csv").map(x => x + ",2013").map(line =>line.split(","))

val in2012 = sc.textFile("MERGED2012_13_PP.csv").map(x => x + ",2012").map(line =>line.split(","))

val in2011 = sc.textFile("MERGED2011_12_PP.csv").map(x => x + ",2011").map(line =>line.split(","))

val in2010 = sc.textFile("MERGED2010_11_PP.csv").map(x => x + ",2010").map(line =>line.split(","))

val in2009 = sc.textFile("MERGED2009_10_PP.csv").map(x => x + ",2009").map(line =>line.split(","))

val in2008 = sc.textFile("MERGED2008_09_PP.csv").map(x => x + ",2008").map(line =>line.split(","))

val in2007 = sc.textFile("MERGED2007_08_PP.csv").map(x => x + ",2007").map(line =>line.split(","))


inHeader: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[3] at map at <console>:25
in2016: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[7] at map at <console>:27
in2015: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[11] at map at <console>:29
in2014: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[15] at map at <console>:31
in2013: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[19] at map at <console>:33
in2012: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[23] at map at <console>:35
in2011: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[27] at map at <console>:37
in2010: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[31] at map at <console>:39
in2009: org.apache.spark.rdd.RDD[Array[String...

## Compile the 10 year RDD.

In [4]:
val scoreCard10Year = sc.union(in2016, in2015, in2014, in2013, in2012, in2011, in2010, in2009, in2008, in2007)
val twoYear = in2016.union(in2015)
scoreCard10Year.take(1)
twoYear.take(1)

scoreCard10Year: org.apache.spark.rdd.RDD[Array[String]] = UnionRDD[44] at union at <console>:36
twoYear: org.apache.spark.rdd.RDD[Array[String]] = UnionRDD[45] at union at <console>:37
res2: Array[Array[String]] = Array(Array(UNITID, OPEID, OPEID6, INSTNM, CITY, STABBR, ZIP, ACCREDAGENCY, INSTURL, NPCURL, SCH_DEG, HCM2, MAIN, NUMBRANCH, PREDDEG, HIGHDEG, CONTROL, ST_FIPS, REGION, LOCALE, LOCALE2, LATITUDE, LONGITUDE, CCBASIC, CCUGPROF, CCSIZSET, HBCU, PBI, ANNHI, TRIBAL, AANAPII, HSI, NANTI, MENONLY, WOMENONLY, RELAFFIL, ADM_RATE, ADM_RATE_ALL, SATVR25, SATVR75, SATMT25, SATMT75, SATWR25, SATWR75, SATVRMID, SATMTMID, SATWRMID, ACTCM25, ACTCM75, ACTEN25, ACTEN75, ACTMT25, ACTMT75, ACTWR25, ACTWR75, ACTCMMID, ACTENMID, ACTMTMID, ACTWRMID, SAT_AVG, SAT_AVG_ALL, PCIP01, PCIP03, PCIP04, PCI...

### Create a header index of the dataset

In [6]:
val header2016 = sc.parallelize(inHeader.take(1))

val header2016Index = header2016.map(_.mkString(",")).flatMap(line => line.split(",")).zipWithIndex

// header2016Index.coalesce(1).saveAsTextFile("index") //Textfiles only made once


header2016: org.apache.spark.rdd.RDD[Array[String]] = ParallelCollectionRDD[52] at parallelize at <console>:27
header2016Index: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[55] at zipWithIndex at <console>:29


## Count of rows for the compiled DoE Scorecard

In [7]:
scoreCard10Year.count()

res4: Long = 74181


### Number of Schools in the DoE dataset

In [8]:
scoreCard10Year.map(x=> x(1)).distinct().count()

res5: Long = 9743


# St Thomas RDD  
This RDD contains all of the St. Thomas results over the years

In [9]:
val ust = scoreCard10Year.filter(x => x(0) contains "174914")

ust: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[60] at filter at <console>:38


In [10]:
//ust.take(1)

In [11]:
ust.count()

res7: Long = 10


In [13]:
val ustHeader = header2016.union(ust)
// ustHeader.map(_.mkString(",")).coalesce(1).saveAsTextFile("ustHeader")

ustHeader: org.apache.spark.rdd.RDD[Array[String]] = UnionRDD[65] at union at <console>:43


# Regional Peers RDD

This RDD filters region 4 (upper midwest/plains), Schools that offer graduate degrees as the highest degree, and a carnegie classification of 4 year, medium size, highly residential. 

In [14]:
val regionalPeers = scoreCard10Year.filter(x => x(18) == "4").filter(x => x(15) == "4").filter(x => x(25) == "13")

regionalPeers: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[71] at filter at <console>:38


In [15]:
regionalPeers.count()

res10: Long = 20


### Average SAT score for peers in the region. Ours is 1204

In [38]:
scoreCard10Year.map(x => x(1899)).map(_.mkString("")).take(1) // I verified index 1899 in all of my load RDDs

res31: Array[String] = Array(2016)


In [16]:
regionalPeers.map(x => x(59)).filter(x => x != "NULL").filter(x => x(1899) == "2016").count()


org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 241, localhost, executor driver): java.lang.StringIndexOutOfBoundsException: String index out of range: 1899

In [19]:
regionalPeers.map(x => x(59)).filter(x => x != "NULL").map(_.toInt).mean()


res14: Double = 1080.7222222222224


# Create an index of the header 

Really had a tough time figuring out how to get the header loaded as just a string. Loading it from the original source results in type RDD[Array[String]], so I ended up using a CSV file that I outputed from the original input. Ended up just needing a **flatMap** to keep it in the correct format. 

# Minnesota Schools  
This rdd contains all of the Minnesota universities for a statewide comparison

In [ ]:
val mnColleges = scoreCard10Year.filter( x => x(5) contains "MN").filter( x => x(1899) contains "2016")

In [ ]:
val mnColLoc = mnColleges.filter(x => x(1899) contains "2016").filter(x => x(36) != "NULL").map(x => x(3) + "," + x(21) + "," + x(22) + "," + x(36))

In [ ]:
// mnColLoc.take(5).foreach(println) // Only needs to run once.

### Create and export RDD to CSV for mapping 

In [ ]:
val mnColLocHead = Array("Name,Lat,Long,AdmRate")
val mnLocRDD = sc.parallelize(mnColLocHead)
val mnColLocFin = mnLocRDD.union(mnColLoc)
//mnColLocFin.coalesce(1).saveAsTextFile("MnSchools") // only run once